In [2]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob
import requests
import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository

In [2]:
connection = sqlite3.connect(settings.db_name,check_same_thread = False)
repo = SQLRepository(connection=connection)


In [4]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_IBM = GarchModel(ticker="IBM", repo=repo, use_new_data=False)

In [6]:
assert gm_IBM.ticker == "IBM"
assert gm_IBM.repo == repo
assert not gm_IBM.use_new_data
assert gm_IBM.model_directory == settings.model_directory

In [7]:
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

In [8]:
# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2019-03-28    0.257926
2019-03-29   -4.352214
2019-04-01    5.895092
2019-04-02    0.402159
2019-04-03   -0.780015
Name: return, dtype: float64

In [9]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2433.82
Distribution:                  Normal   AIC:                           4875.65
Method:            Maximum Likelihood   BIC:                           4895.28
                                        No. Observations:                 1000
Date:                Wed, Apr 12 2023   Df Residuals:                      999
Time:                        10:57:37   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1184  7.694e-02      1.538      0.124 [-3.244e-02,  0.269]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.1821      0.242      0.754      0.451     [ -0.291,  0.655]
alpha[1]       0.0434  2.192e-02      1.980  4.765e-02 [4.487e-04,8.636e-02]
beta[1]        0.9363  4.727e-02     19.806  2.620e-87     [  0.844,  1.029]
============================================================================

Covariance estimator: robust
"""

In [10]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-04-12T00:00:00': 2.1223784293917447,
 '2023-04-13T00:00:00': 2.14359654731172,
 '2023-04-14T00:00:00': 2.164181671150938,
 '2023-04-17T00:00:00': 2.184160259127017,
 '2023-04-18T00:00:00': 2.2035571217161554}

In [11]:
model_directory = settings.model_directory
ticker = "SHOPERSTOP"
timestamp = pd.Timestamp.now().isoformat()

filepath = os.path.join(model_directory,f"{timestamp}_{ticker}.pkl")
filepath

'models/2023-04-12T10:58:02.599949_SHOPERSTOP.pkl'

In [12]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'models/2023-04-12T10:58:15.541266_SHOPERSTOP.BSE.pkl'

In [3]:
url = "http://localhost:8008/hello"
response = requests.get(url=url)

print("response code:", response.status_code)
response.json()

response code: 200


{'message': 'Hello World!'}

In [4]:
from main import build_model

# Instantiate `GarchModel` with function
model_shop = build_model(ticker="SHOPERSTOP.BSE", use_new_data=False)
model_shop

In [5]:
url = "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker": "IBM",
    "use_new_data" : True,
    "n_observations": 2000,
    "p":1,
    "q" : 1
    
}
# Response of post request
response = requests.post(url=url,json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'IBM',
 'use_new_data': True,
 'n_observations': 2000,
 'p': 1,
 'q': 1,
 'success': True,
 'message': "Trained and saved 'models/2023-04-12T11:31:32.368947_IBM.pkl' Metrics: AIC 7137.549002867772, BIC 7159.952612705941."}

In [6]:
url = "http://localhost:8008/predict"
# Data to send to path
json = {
    "ticker":"SHOPERSTOP.BSE",
    "n_days": 5
}
# Response of post request
response = requests.post(url=url,json=json)
response.json()

{'ticker': 'SHOPERSTOP.BSE',
 'n_days': 5,
 'success': True,
 'forecast': {'2023-04-12T00:00:00': 2.1223784293917447,
  '2023-04-13T00:00:00': 2.14359654731172,
  '2023-04-14T00:00:00': 2.164181671150938,
  '2023-04-17T00:00:00': 2.184160259127017,
  '2023-04-18T00:00:00': 2.2035571217161554},
 'message': ''}